# 타이타닉 생존자 예측 - Pro_4 (Pipeline + No Target Leakage)

## 🎯 Pro_4 핵심 전략
1. **Target Leakage 완전 방지**: sklearn Pipeline + Custom Transformer
   - `DeckSurviveRatioEncoder`: CV 각 Fold에서 Train만으로 생존율 계산
   - `PriceBinEncoder`: CV 각 Fold에서 Train만으로 bin 경계 계산
2. **Price Winsorizing**: Train IQR 기준
3. **KNN Imputer**: Train fit → Val/Test transform
4. **Deck 결측치**: Train 매핑 기준

In [20]:
# ============================================================
# 1. 라이브러리 임포트
# ============================================================
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import KNNImputer
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from xgboost import XGBClassifier

import optuna
from optuna.samplers import TPESampler

import warnings
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

print('✅ 라이브러리 임포트 완료')

✅ 라이브러리 임포트 완료


In [21]:
# ============================================================
# 2. Custom Transformers 정의 (Target Leakage 방지)
# ============================================================

class DeckSurviveRatioEncoder(BaseEstimator, TransformerMixin):
    """Deck별 생존율을 fit 시점의 y(target)로만 계산하여 적용"""
    
    def __init__(self, deck_col='Deck'):
        self.deck_col = deck_col
        self.deck_ratio_map_ = None
        self.default_ratio_ = None
        
    def fit(self, X, y=None):
        if y is None:
            raise ValueError("y must be provided for fit")
        
        # X와 y를 DataFrame으로 합쳐서 계산
        df = X.copy() if isinstance(X, pd.DataFrame) else pd.DataFrame(X)
        df['_target'] = y.values if hasattr(y, 'values') else y
        
        # Deck별 생존율 계산
        self.deck_ratio_map_ = df.groupby(self.deck_col)['_target'].mean().to_dict()
        self.default_ratio_ = df['_target'].mean()
        
        return self
    
    def transform(self, X):
        X = X.copy() if isinstance(X, pd.DataFrame) else pd.DataFrame(X)
        X['Deck_survive_ratio'] = X[self.deck_col].map(self.deck_ratio_map_)
        X['Deck_survive_ratio'] = X['Deck_survive_ratio'].fillna(self.default_ratio_)
        return X


class PriceBinEncoder(BaseEstimator, TransformerMixin):
    """Price를 fit 시점의 데이터로만 qcut 경계 계산하여 binning"""
    
    def __init__(self, price_col='Price', n_bins=5):
        self.price_col = price_col
        self.n_bins = n_bins
        self.bin_edges_ = None
        
    def fit(self, X, y=None):
        df = X.copy() if isinstance(X, pd.DataFrame) else pd.DataFrame(X)
        _, self.bin_edges_ = pd.qcut(df[self.price_col], self.n_bins, 
                                     labels=False, retbins=True, duplicates='drop')
        return self
    
    def transform(self, X):
        X = X.copy() if isinstance(X, pd.DataFrame) else pd.DataFrame(X)
        X['PriceBin'] = pd.cut(X[self.price_col], bins=self.bin_edges_, 
                               labels=range(len(self.bin_edges_)-1), include_lowest=True)
        X['PriceBin'] = X['PriceBin'].astype(float).fillna(2).astype(int)  # 중간값 fallback
        return X


class FeatureSelector(BaseEstimator, TransformerMixin):
    """최종 Feature만 선택"""
    
    def __init__(self, features):
        self.features = features
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.features]


print('✅ Custom Transformers 정의 완료')

✅ Custom Transformers 정의 완료


In [22]:
# ============================================================
# 3. 데이터 불러오기 및 기본 전처리 (Target 미사용)
# ============================================================
base_path = r'C:/Users/user/github/DataScience/scikit-learn/scikit-learn/data/titanic'
train_full = pd.read_csv(f'{base_path}/train.csv')
test_df = pd.read_csv(f'{base_path}/test.csv')

test_passenger_ids = test_df['PassengerId'].copy()

# Train/Validation 분리
train_df, val_df = train_test_split(train_full, test_size=0.15, random_state=42, stratify=train_full['Survived'])
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

print(f'Train: {train_df.shape}, Val: {val_df.shape}, Test: {test_df.shape}')

Train: (757, 12), Val: (134, 12), Test: (418, 11)


In [23]:
# ============================================================
# 4. 기본 Feature Engineering (Target 사용 안 함)
# ============================================================

def basic_preprocessing(df):
    df = df.copy()
    
    # Title 추출
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\\.', expand=False)
    df.loc[df.Title.isin(['Mrs', 'Mme']), 'Title'] = 'Mrs'
    df.loc[df.Title.isin(['Ms', 'Miss', 'Mlle']), 'Title'] = 'Miss'
    df.loc[(~df.Title.isin(['Mrs', 'Miss', 'Mr', 'Master']) & df.Sex.eq('male')), 'Title'] = 'Mr'
    df.loc[(~df.Title.isin(['Mrs', 'Miss', 'Mr', 'Master']) & df.Sex.eq('female')), 'Title'] = 'Mrs'
    
    # Ticket 정보
    df['Ticket_nr'] = [i[-1] for i in df.Ticket.str.split()]
    
    # Deck 추출
    df['Deck'] = df.Cabin.str[0]
    df.loc[df.Deck.eq('T'), 'Deck'] = 'A'
    
    # Sex, Embarked 인코딩
    df['Sex'] = (df['Sex'] == 'male').astype(int)
    df['Embarked'] = df['Embarked'].fillna('S').map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
    
    # Family Features
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    
    return df

train_df = basic_preprocessing(train_df)
val_df = basic_preprocessing(val_df)
test_df = basic_preprocessing(test_df)

print('✅ 기본 전처리 완료')

✅ 기본 전처리 완료


In [24]:
# ============================================================
# 5. Price 계산 (Train Ticket 통계 기준)
# ============================================================

ticket_dict = train_df.groupby('Ticket_nr').Name.count().to_dict()

def calc_price(df, ticket_map):
    df = df.copy()
    df['Passengers_ticket'] = df.Ticket_nr.map(ticket_map).fillna(1)
    df['Price'] = (df.Fare / df.Passengers_ticket).round(2)
    df.loc[df['Price'] == 0, 'Price'] = np.nan
    return df

train_df = calc_price(train_df, ticket_dict)
val_df = calc_price(val_df, ticket_dict)
test_df = calc_price(test_df, ticket_dict)

In [25]:
# ============================================================
# 6. Price Winsorizing (Train IQR 기준)
# ============================================================

price_bounds = {}
for pc in [1, 2, 3]:
    prices = train_df.loc[(train_df.Pclass == pc) & train_df.Price.notna(), 'Price']
    Q1, Q3 = prices.quantile(0.25), prices.quantile(0.75)
    IQR = Q3 - Q1
    price_bounds[pc] = (max(0, Q1 - 1.5*IQR), Q3 + 1.5*IQR)

def winsorize(df, bounds):
    df = df.copy()
    for pc, (lo, hi) in bounds.items():
        m = df.Pclass == pc
        df.loc[m & (df.Price < lo), 'Price'] = lo
        df.loc[m & (df.Price > hi), 'Price'] = hi
    return df

train_df = winsorize(train_df, price_bounds)
val_df = winsorize(val_df, price_bounds)
test_df = winsorize(test_df, price_bounds)

In [26]:
# ============================================================
# 7. Deck & Price & Age 결측치 처리 (Train 기준 KNN)
# ============================================================

# Deck 기본 처리
default_deck = {1: 'C', 2: 'F', 3: 'G'}
deck_level = {'G': 1, 'F': 2, 'E': 3, 'D': 4, 'C': 5, 'B': 6, 'A': 7}

def fill_deck_and_encode(df):
    df = df.copy()
    for idx in df[df.Deck.isna()].index:
        df.loc[idx, 'Deck'] = default_deck.get(int(df.loc[idx, 'Pclass']), 'G')
    df['Deck'] = df['Deck'].map(deck_level).fillna(1).astype(int)
    return df

train_df = fill_deck_and_encode(train_df)
val_df = fill_deck_and_encode(val_df)
test_df = fill_deck_and_encode(test_df)

# Title 인코딩
title_map = {'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3}
for df in [train_df, val_df, test_df]:
    df['Title'] = df['Title'].map(title_map).fillna(0).astype(int)

# Price 결측치 (Train 평균)
price_mean_train = train_df.groupby('Pclass').Price.mean().to_dict()
for df in [train_df, val_df, test_df]:
    for idx in df[df.Price.isna()].index:
        df.loc[idx, 'Price'] = price_mean_train.get(int(df.loc[idx, 'Pclass']), 7.5)

# Age KNN (Train fit)
knn_cols = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Price', 'Embarked', 'Deck', 'Title', 'Age']
imputer = KNNImputer(n_neighbors=7)

train_imp = pd.DataFrame(imputer.fit_transform(train_df[knn_cols]), columns=knn_cols)
train_df['Age'] = train_imp['Age'].values

val_imp = pd.DataFrame(imputer.transform(val_df[knn_cols]), columns=knn_cols)
val_df['Age'] = val_imp['Age'].values

test_imp = pd.DataFrame(imputer.transform(test_df[knn_cols]), columns=knn_cols)
test_df['Age'] = test_imp['Age'].values

print('✅ Deck/Price/Age 결측치 처리 완료')

✅ Deck/Price/Age 결측치 처리 완료


In [27]:
# ============================================================
# 8. AgeBin 생성
# ============================================================

def age_bin(age):
    if age < 10: return 0
    elif age < 25: return 1
    elif age < 40: return 2
    elif age < 55: return 3
    else: return 4

for df in [train_df, val_df, test_df]:
    df['AgeBin'] = df['Age'].apply(age_bin)
    df['IsChild'] = (df['AgeBin'] == 0).astype(int)

In [28]:
# ============================================================
# 9. Pipeline 구성 (Deck_survive_ratio, PriceBin을 CV 내에서 계산)
# ============================================================

# 최종 사용할 Feature (Deck_survive_ratio, PriceBin 제외 - Pipeline에서 생성)
base_features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked', 'Title', 
                 'AgeBin', 'Deck', 'FamilySize', 'IsAlone', 'IsChild', 'Price']

final_features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked', 'Title', 
                  'AgeBin', 'Deck', 'FamilySize', 'IsAlone', 'IsChild', 
                  'Deck_survive_ratio', 'PriceBin']

X_train = train_df[base_features].copy()
y_train = train_df['Survived'].astype(int)
X_val = val_df[base_features].copy()
y_val = val_df['Survived'].astype(int)
X_test = test_df[base_features].copy()

# Preprocessing Pipeline
preprocessor = Pipeline([
    ('deck_ratio', DeckSurviveRatioEncoder(deck_col='Deck')),
    ('price_bin', PriceBinEncoder(price_col='Price', n_bins=5)),
    ('selector', FeatureSelector(final_features))
])

print(f'Base Features: {base_features}')
print(f'Final Features: {final_features}')

Base Features: ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked', 'Title', 'AgeBin', 'Deck', 'FamilySize', 'IsAlone', 'IsChild', 'Price']
Final Features: ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked', 'Title', 'AgeBin', 'Deck', 'FamilySize', 'IsAlone', 'IsChild', 'Deck_survive_ratio', 'PriceBin']


In [29]:
# ============================================================
# 10. Cross-Validation with Pipeline (Target Leakage Free)
# ============================================================

def evaluate_model_cv(model, X, y, preprocessor, cv=5):
    """CV 각 Fold에서 preprocessor를 fit하여 target leakage 방지"""
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
    scores = []
    
    for train_idx, val_idx in skf.split(X, y):
        X_tr, X_vl = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_vl = y.iloc[train_idx], y.iloc[val_idx]
        
        # 각 Fold에서 preprocessor를 새로 fit!
        prep = preprocessor.__class__(preprocessor.steps)
        X_tr_processed = prep.fit_transform(X_tr, y_tr)
        X_vl_processed = prep.transform(X_vl)
        
        # 모델 학습
        model_clone = model.__class__(**model.get_params())
        model_clone.fit(X_tr_processed, y_tr)
        
        # 평가
        pred = model_clone.predict(X_vl_processed)
        scores.append(accuracy_score(y_vl, pred))
    
    return np.mean(scores), np.std(scores)


# 모델 정의
models = {
    'XGB': XGBClassifier(n_estimators=200, max_depth=4, learning_rate=0.1, 
                         reg_alpha=1.0, subsample=0.8, random_state=42,
                         use_label_encoder=False, eval_metric='logloss', n_jobs=-1),
    'RF': RandomForestClassifier(n_estimators=200, max_depth=6, min_samples_split=5, 
                                  random_state=42, n_jobs=-1),
    'LogReg': LogisticRegression(C=0.5, max_iter=1000, random_state=42),
    'SVC': SVC(probability=True, C=1.0, kernel='rbf', random_state=42),
    'MLP': MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=1000, 
                          random_state=42, early_stopping=True)
}

print('📊 Cross-Validation (No Target Leakage):')
for name, model in models.items():
    mean_acc, std_acc = evaluate_model_cv(model, X_train, y_train, preprocessor, cv=5)
    print(f'  {name}: {mean_acc:.4f} ± {std_acc:.4f}')

📊 Cross-Validation (No Target Leakage):
  XGB: 0.8402 ± 0.0177
  RF: 0.8414 ± 0.0344
  LogReg: 0.8270 ± 0.0168
  SVC: 0.8362 ± 0.0204
  MLP: 0.8296 ± 0.0241


In [30]:
# ============================================================
# 11. 최종 모델 학습 (전체 Train 사용)
# ============================================================

# 전체 Train으로 preprocessor fit
X_train_final = preprocessor.fit_transform(X_train, y_train)
X_val_final = preprocessor.transform(X_val)
X_test_final = preprocessor.transform(X_test)

# 모델 학습
trained_models = {}
for name, model in models.items():
    model.fit(X_train_final, y_train)
    trained_models[name] = model

print('\n📊 Validation Set 성능:')
for name, model in trained_models.items():
    acc = accuracy_score(y_val, model.predict(X_val_final))
    print(f'  {name}: {acc:.4f}')


📊 Validation Set 성능:
  XGB: 0.8134
  RF: 0.8209
  LogReg: 0.8134
  SVC: 0.8134
  MLP: 0.8134


In [31]:
# ============================================================
# 12. 앙상블
# ============================================================

estimators = [(name, model) for name, model in trained_models.items()]

voting = VotingClassifier(estimators=estimators, voting='soft')
voting.fit(X_train_final, y_train)
print(f'Voting Val Acc: {voting.score(X_val_final, y_val):.4f}')

stacking = StackingClassifier(estimators=estimators, 
                               final_estimator=LogisticRegression(C=0.1, max_iter=1000),
                               cv=5, n_jobs=-1)
stacking.fit(X_train_final, y_train)
print(f'Stacking Val Acc: {stacking.score(X_val_final, y_val):.4f}')

Voting Val Acc: 0.8209
Stacking Val Acc: 0.8209


In [32]:
# ============================================================
# 13. 최종 제출 (Train+Val로 재학습)
# ============================================================
import os

# Train + Val 합치기
X_full = pd.concat([X_train, X_val], ignore_index=True)
y_full = pd.concat([y_train, y_val], ignore_index=True)

# 전체 데이터로 preprocessor 재fit
X_full_processed = preprocessor.fit_transform(X_full, y_full)
X_test_processed = preprocessor.transform(X_test)

# Voting, Stacking 재학습
voting.fit(X_full_processed, y_full)
stacking.fit(X_full_processed, y_full)

output_path = r'C:/Users/user/github/DataScience/scikit-learn/scikit-learn/Submission'
os.makedirs(output_path, exist_ok=True)

for name, model in [('Pro4_Voting_NoLeak', voting), ('Pro4_Stacking_NoLeak', stacking)]:
    pred = model.predict(X_test_processed).astype(int)
    pd.DataFrame({'PassengerId': test_passenger_ids, 'Survived': pred}).to_csv(
        f'{output_path}/submission_{name}.csv', index=False)
    print(f'✅ submission_{name}.csv 저장 완료 (생존: {pred.sum()}/{len(pred)})')

✅ submission_Pro4_Voting_NoLeak.csv 저장 완료 (생존: 154/418)
✅ submission_Pro4_Stacking_NoLeak.csv 저장 완료 (생존: 149/418)
